In [1]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))
    
from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data, rel_disp, third_moment
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3,  x_log_of_pn
from MPyDATA_examples.utils.show_plot import show_plot
import numpy as np
import matplotlib
import pathlib

In [3]:
def compute_and_plot(psi_coord, grid_layout, n_bins, GC_max, opt_set, plots, filename):
    results, setup = compute_figure_data(
        psi_coord=psi_coord, 
        grid_layouts=(grid_layout,),
        nr=n_bins,
        GC_max = GC_max,
        opt_set=opt_set
    )    
    colors = ['red', 'green', 'purple', 'blue', 'orange']
    matplotlib.rcParams.update({'font.size': 16})
    for coord in results.keys():
        out_steps = results[coord]['grid']['out_steps']
        dt = results[coord]['grid']['dt']
        plotter = Plotter(setup, plots=plots)
        for opt_i, opts in enumerate(results[coord]['numerical'].keys()):
            plot_data = results[coord]['numerical'][opts]
            row = []
            for i in range(len(out_steps)):
                mnorm = setup.mixing_ratios[i]
                t = out_steps[i] * dt
                linewidth = 1+1.5*i/len(out_steps)
                if opt_i == 0:
                    plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        results[coord]['analytical'][i], 
                        bin_boundaries = results[coord]['grid']['rh'], 
                        label='analytical', 
                        mnorm=mnorm,
                        color='black',
                        linewidth = linewidth
                    )
                    
                    
                str_repl = [["'n_iters': 1","upwind"],
                            ["'n_iters': 2","MPDATA 2 iterations"],
                            ["'n_iters': 3","MPDATA 3 iterations"],
                            ["'",""],
                            [": True",""],
                            ["_", " "],
                            ["{",""],["}",""],[","," "]]                            
                for repl in str_repl:
                    opts = opts.replace(repl[0], repl[1])
                plotter.pdf_histogram(
                    results[coord]['grid']['r'], 
                    plot_data[i], 
                    label=opts, 
                    bin_boundaries=results[coord]['grid']['rh'], 
                    linewidth = linewidth,
                    mnorm=mnorm, color = colors[opt_i]
                )
                
                dp_dr = psi_coord.dx_dr(results[coord]['grid']['r'])
                numeric_rel_d = rel_disp(results[coord]['grid']['rh'], plot_data[i]/dp_dr, psi_coord)
                analytic_rel_d = rel_disp(results[coord]['grid']['rh'], results[coord]['analytical'][i]/dp_dr, psi_coord)
                print("analytic rd", analytic_rel_d)
                print("numeric rd", numeric_rel_d)
                dispersion_ratio = (numeric_rel_d / analytic_rel_d - 1)*100
                print(f"dispersion ratio excess {opts}: {dispersion_ratio.to(setup.si.dimensionless).magnitude:.2g} %")
                print("\n")
                                
                numeric_mass = third_moment(results[coord]['grid']['rh'], plot_data[i]/dp_dr, psi_coord, normalised=False)
                analytic_mass = third_moment(results[coord]['grid']['rh'], results[coord]['analytical'][i]/dp_dr, psi_coord, normalised=False)
                mass_ratio = (numeric_mass / analytic_mass - 1) * 100
                print(f"mass ratio excess: {mass_ratio.to(setup.si.dimensionless).magnitude:.2g} %")
                print("\n")
                
#                 row.append((mnorm.magnitude, f"{mass_ratio.to(setup.si.dimensionless).magnitude:.2g} %"))
                   
#             if opts in excess_mass_ratios:
#                 assert excess_mass_ratios[opts] == row
#             else:
#                 excess_mass_ratios[opts] = row
        print(f"grid_layout: {coord}")
        show_plot(filename)

In [4]:
plot_setup = {"psi_coord":x_p2(), "grid_layout":x_log_of_pn(r0=1, base=2), "GC_max":.26,"n_bins":75}
plot_setup2 = {"psi_coord":x_id(), "grid_layout":x_id(), "GC_max":.07,"n_bins":75}
excess_mass_ratios={}
excess_dispersion_ratios={}
numeric_dispersions={}
analytic_dispersions={}

In [5]:
compute_and_plot(**plot_setup, opt_set=({"n_iters":1},), plots=('n','m'), filename = 'fig_upwinda.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    8.0s finished


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.15654148303172583 dimensionless
dispersion ratio excess upwind: 24 %


mass ratio excess: 5.5 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.11780165417654583 dimensionless
dispersion ratio excess upwind: 71 %


mass ratio excess: 4.4 %


grid_layout: x_log_of_pn


In [6]:
compute_and_plot(**plot_setup2, opt_set=({"n_iters":1},), plots=['n','m'],  filename = 'fig_upwindb.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    9.0s finished


analytic rd 0.3569668694267919 dimensionless
numeric rd 0.3569668694267919 dimensionless
dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


analytic rd 0.12553338250327037 dimensionless
numeric rd 0.14969083028424135 dimensionless
dispersion ratio excess upwind: 19 %


mass ratio excess: 0.89 %


analytic rd 0.06717971636291314 dimensionless
numeric rd 0.09911854639733908 dimensionless
dispersion ratio excess upwind: 48 %


mass ratio excess: 0.41 %


grid_layout: x_id


In [7]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':3}), plots=['n'], filename = 'fig_mpdatas.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   58.7s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   58.7s finished


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.15654148303172583 dimensionless
dispersion ratio excess upwind: 24 %


mass ratio excess: 5.5 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.11780165417654583 dimensionless
dispersion ratio excess upwind: 71 %


mass ratio excess: 4.4 %


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess MPDATA 2 iterations: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.14169886799513856 dimensionless
dispersion ratio excess MPDATA 2 iterations: 13 %


mass ratio excess: 2 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.09810918714188281 dimensionless
dispersion ratio excess MPDATA 2 iterations: 42 %


mass ratio excess: 0.82 %


analytic rd 0.3569974994072641

In [8]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True}), plots=['n'], filename = 'fig_iga.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   41.5s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   41.5s finished


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.15654148303172583 dimensionless
dispersion ratio excess upwind: 24 %


mass ratio excess: 5.5 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.11780165417654583 dimensionless
dispersion ratio excess upwind: 71 %


mass ratio excess: 4.4 %


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess MPDATA 2 iterations: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.14169886799513856 dimensionless
dispersion ratio excess MPDATA 2 iterations: 13 %


mass ratio excess: 2 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.09810918714188281 dimensionless
dispersion ratio excess MPDATA 2 iterations: 42 %


mass ratio excess: 0.82 %


analytic rd 0.3569974994072641

In [9]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True,'flux_corrected_transport':True}), plots=['n'], filename = 'fig_fct.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:  1.0min remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:  1.0min finished


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.15654148303172583 dimensionless
dispersion ratio excess upwind: 24 %


mass ratio excess: 5.5 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.11780165417654583 dimensionless
dispersion ratio excess upwind: 71 %


mass ratio excess: 4.4 %


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess MPDATA 2 iterations: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.14169886799513856 dimensionless
dispersion ratio excess MPDATA 2 iterations: 13 %


mass ratio excess: 2 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.09810918714188281 dimensionless
dispersion ratio excess MPDATA 2 iterations: 42 %


mass ratio excess: 0.82 %


analytic rd 0.3569974994072641

In [10]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'flux_corrected_transport':True}), plots=['n'], filename = 'fig_fct.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:  1.2min remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:  1.2min finished


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.15654148303172583 dimensionless
dispersion ratio excess upwind: 24 %


mass ratio excess: 5.5 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.11780165417654583 dimensionless
dispersion ratio excess upwind: 71 %


mass ratio excess: 4.4 %


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess MPDATA 2 iterations: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.14169886799513856 dimensionless
dispersion ratio excess MPDATA 2 iterations: 13 %


mass ratio excess: 2 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.09810918714188281 dimensionless
dispersion ratio excess MPDATA 2 iterations: 42 %


mass ratio excess: 0.82 %


analytic rd 0.3569974994072641

In [11]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':3},{'n_iters':3,'third_order_terms':True}), plots=['n'], filename = 'fig_tot.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:  1.5min finished


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.15654148303172583 dimensionless
dispersion ratio excess upwind: 24 %


mass ratio excess: 5.5 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.11780165417654583 dimensionless
dispersion ratio excess upwind: 71 %


mass ratio excess: 4.4 %


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess MPDATA 3 iterations: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.13860642484602037 dimensionless
dispersion ratio excess MPDATA 3 iterations: 10 %


mass ratio excess: 1.5 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.09312885227027101 dimensionless
dispersion ratio excess MPDATA 3 iterations: 35 %


mass ratio excess: 0.15 %


analytic rd 0.35699749940726

In [12]:
compute_and_plot(**plot_setup, opt_set=(
    {'n_iters':1},
    {'n_iters':2},
    {'n_iters':3,'infinite_gauge':True, 'flux_corrected_transport':True,'third_order_terms':True},
), plots=['n'], filename='fig_multiopt.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:  1.7min finished


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.15654148303172583 dimensionless
dispersion ratio excess upwind: 24 %


mass ratio excess: 5.5 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.11780165417654583 dimensionless
dispersion ratio excess upwind: 71 %


mass ratio excess: 4.4 %


analytic rd 0.35699749940726416 dimensionless
numeric rd 0.35699749940726416 dimensionless
dispersion ratio excess MPDATA 2 iterations: 0 %


mass ratio excess: 0 %


analytic rd 0.125866239312808 dimensionless
numeric rd 0.14169886799513856 dimensionless
dispersion ratio excess MPDATA 2 iterations: 13 %


mass ratio excess: 2 %


analytic rd 0.06898528061217586 dimensionless
numeric rd 0.09810918714188281 dimensionless
dispersion ratio excess MPDATA 2 iterations: 42 %


mass ratio excess: 0.82 %


analytic rd 0.3569974994072641

In [13]:
def compare_refdata(data, name, generate=False):
    path = pathlib.Path(os.getcwd()).joinpath(f"{name}_refdata.txt")
    datastr=str(data)
    if generate:
        with open(path, 'w+') as file: 
            file.write(datastr)
    else:
        with open(path, 'r') as file:
            assert file.read()==datastr
            print(file)

In [14]:
compare_refdata(excess_mass_ratios, name="mass ratios", generate=True)

In [15]:
def make_textable(data, generate=False, print_tab=False):
    latex_data = r"\hline" + " Variant  & Elapsed Real Time (wrt upwind) " + r"\\ \hline" + "\n"
    for opt, value in zip(data["opts"], data["values"]):
        latex_data += r"\hline" + f" {opt} & {value} " + r"\\ \hline" + "\n"
    latex_start = r"\begin{table}[]" + "\n" + r"\begin{tabular}{| l | l |}" + "\n"
    latex_end = "\end{tabular} \n \end{table}"
    latex_table = latex_start + latex_data + latex_end
    if print_tab:
        print(latex_table)
    with open(pathlib.Path(__file__).parent.joinpath("wall_time_textable.txt"), "w+" if generate else "r") as f:
        if generate:
            f.write(latex_table)